In [97]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd
from math import radians, degrees, atan2, sqrt

In [5]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[300:] 

In [ ]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoCropper(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    
    data: DataFrame
    Mouse moving location, by pixel
    
    !!! We are assuming that the mouse at least reaches the boundary once
    '''
    x = sorted(data['x'], reverse = True)
    y = sorted(data['y'], reverse = True)
    
    # Try to exclude outliers
    for i in range(10):
        xMax = round(x[i + 1]) if x[i] - x[i + 1] >= 1 else round(x[i])
        
        xMin = round(x[-i-1]) if x[-i] - x[-i-1] >= 1 else round(x[-i])
            
        yMax = round(y[i + 1]) if y[i] - y[i + 1] >= 1 else round(y[i])

        yMin = round(y[-i-1]) if y[-i] - y[-i-1] >= 1 else round(y[-i])
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
    
    return xMin, yMin, lengthX, lengthY



def dataModifier(data):
    '''
    Task: Modify the location data based on the cropped video
    
    data: DataFrame
    '''
    
    x, y, lengthX, lengthY = getCoord(data)
    filtered = data.copy(deep = True)
    filtered = filtered[(filtered['y']>x) & (filtered['x']>y)]
    filtered['x'] = filtered['x'] - y
    filtered['y'] = filtered['y'] - x
    filtered['coords'] = list(range(1,len(filtered)+1))
    filtered.index = list(range(1,len(filtered)+1))
    
    return filtered

In [ ]:
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.


In [96]:
def quadrant(a, b): 
    '''
    Task: Check if two points are in the same quadrant
    '''
    if ((a[0] > 0 and b[0] > 0) or (a[0] < 0 and b[0] < 0)) and ((a[1] > 0 and b[1] > 0) or (a[1] < 0 and b[1] < 0)):
            return True
    else:
        return False
    
def getAngle(a, b, c):
    '''
    Task: Calculate angle between three points. 
    
    PARAMETERS
    -----------
    a, b, c: tuple
        Three points in the coordinate system
    '''
    cb = atan2(c[1]-b[1], c[0]-b[0])
    ab = atan2(a[1]-b[1], a[0]-b[0])

    # If they are in the same quadrant: acute angle
    if quadrant(c, b) and quadrant(a, b):
        angle = degrees(cb - ab)
    else:
    # Obtuse angle or in different quadrant
        rad360 = radians(360)
        cb = rad360 + cb if cb < 0 else cb
        ab = rad360 + ab if ab < 0 else ab
        angle = abs(degrees(cb - ab))
    return angle + 360 if angle < 0 else angle

def getDist(a, b):
    '''
    Task: Calculate distance
    '''
    dist = sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)  
    return dist  

In [103]:
def calDist_Angle(data, enclosure):
    '''
    Task: Calculate distance and angle between enclosure and defeated mouse
    reference point is the center point of head of tail regarding Angle
    head point for Distance
    
    Parameters
    ----------
    data: DataFrame
        Read location data
    
    enclosure: tuple
        enclosure's coordinate, ex. (68,204)
    '''
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    # Head--------------------Tail-----------
    mid = (p1.x + p2.x) / 2, (p1.y + p2.y) / 2
    # Avoid Pandas KeyError
    mid = np.array(mid)
    x = np.array(p1.x)
    y = np.array(p1.y)
    
    angle = []
    dist = []
    for i in range(len(p1)):
        # Set middle point of the mouse to be (0,0), then calculate the corresponding relative converted coordinates
        # Order: head relative location, central point, enclosure relative location
        angle.append(getAngle((x[i] - mid[0][i], y[i] - mid[1][i]),(0, 0), (enclosure[0] - mid[0][i], enclosure[1] - mid[1][i])))
        dist.append(getDist((mid[0][i], mid[1][i]), enclosure))
    return angle, dist

In [114]:
for i, value in enumerate(calDist_Angle(data, (68,204))[1]):
    if 

TypeError: '>' not supported between instances of 'list' and 'int'

In [129]:
# import the necessary packages
import numpy as np
import cv2
 
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped, M



In [130]:
pts = np.array([(28,39),(74,387),(364,23),(377,336)])
img = cv2.imread("test2.jpg")
cv2.imwrite("test10.jpg",four_point_transform(img, pts)[0])

True

In [180]:
np.rot90(pd.concat([p1,pd.DataFrame([1]*len(p1))], axis = 1).values)[::-1]

array([[364.71005243, 381.16299605,  -6.38130188, ..., 238.62146091,
        236.24977016, 235.43116379],
       [483.99807835, 484.94363976, 474.44571853, ...,  65.6972239 ,
         66.07708442,  64.19483757],
       [  1.        ,   1.        ,   1.        , ...,   1.        ,
          1.        ,   1.        ]])

In [182]:
np.dot(four_point_transform(img, pts)[1].reshape(3,3),np.rot90(pd.concat([p1,pd.DataFrame([1]*len(p1))], axis = 1).values)[::-1])

array([[239.51141475, 253.5844412 , -79.2430577 , ..., 178.49255253,
        176.40512073, 175.91400833],
       [410.02155988, 411.55928776, 385.81028164, ...,  32.66322184,
         32.90061134,  31.19195525],
       [  0.76872823,   0.76286879,   0.89830812, ...,   0.90363667,
          0.90436829,   0.90506166]])

In [157]:
(pd.concat([p1,pd.DataFrame([1]*len(p1))], axis = 1).values.shape)

(3481, 3)

In [147]:
four_point_transform(img, pts)[1][0]

array([  0.86189764,  -0.113929  , -19.68990303])

In [164]:
np.dot(four_point_transform(img, pts)[1][0] , pd.concat([p1,pd.DataFrame([1]*len(p1))], axis = 1).values[0])

410.02155987844924

In [148]:
0.86189*364 - 0.113929*483.998078 - 19.6899

238.896642971538

In [151]:
for i in pd.concat([p1,pd.DataFrame([1]*len(p1))], axis = 1).values:
    print(i)

[364.71005243 483.99807835   1.        ]
[381.16299605 484.94363976   1.        ]
[ -6.38130188 474.44571853   1.        ]
[ -6.6551199  474.23213601   1.        ]
[411.75760938 482.76291656   1.        ]
[381.01685047 484.34515381   1.        ]
[ -6.08248425 474.07858717   1.        ]
[ -6.84143162 474.17822134   1.        ]
[ 27.88470085 480.38709021   1.        ]
[ -6.74255276 473.95055699   1.        ]
[381.25194132 484.49326229   1.        ]
[ -7.20973778 473.79029846   1.        ]
[ -6.61289692 474.15190768   1.        ]
[ -6.61118889 473.97455478   1.        ]
[ -5.95636654 474.06735861   1.        ]
[ -6.56460667 473.99111366   1.        ]
[ -5.98256302 473.62858129   1.        ]
[ -6.4928627  473.80507755   1.        ]
[380.22138904 483.02143669   1.        ]
[ -6.46379566 474.20114613   1.        ]
[ -6.32737541 474.15133274   1.        ]
[381.00678837 483.51747036   1.        ]
[381.44156182 483.25132895   1.        ]
[ -6.34524632 474.16240489   1.        ]
[ -6.61620235 47

[340.70264512  76.51885414   1.        ]
[339.84683457  73.0507865    1.        ]
[337.73631835  72.07550287   1.        ]
[334.33480072  68.45597172   1.        ]
[325.82475996  69.65892327   1.        ]
[317.96582985  69.04478061   1.        ]
[311.17721939  66.74008131   1.        ]
[307.87840022  67.98128758   1.        ]
[299.13782454  56.78386784   1.        ]
[297.93148708  50.51121855   1.        ]
[292.04984584  45.87401772   1.        ]
[283.61817217  42.03618765   1.        ]
[280.67776012  43.99589157   1.        ]
[277.52688503  55.54060793   1.        ]
[274.63072014  67.53334692   1.        ]
[262.83376884  71.46936989   1.        ]
[259.2644915   75.50162601   1.        ]
[247.38677073  77.52413273   1.        ]
[243.83209123  79.31091022   1.        ]
[232.23428297  81.15447617   1.        ]
[230.17931032  78.6158638    1.        ]
[228.13384476  77.36621284   1.        ]
[214.91227794  76.5107975    1.        ]
[212.58988219  78.45192623   1.        ]
[203.48096085  8

[105.52336216 357.76065004   1.        ]
[107.3572855  357.93585241   1.        ]
[109.83953905 358.19237566   1.        ]
[111.3797946  358.13114047   1.        ]
[114.70062065 358.06207633   1.        ]
[118.00226068 357.67357373   1.        ]
[121.83631039 358.87915778   1.        ]
[126.05577564 360.30715418   1.        ]
[134.42479324 348.90736103   1.        ]
[135.24669409 348.68532848   1.        ]
[132.67392755 348.09289551   1.        ]
[126.98213816 352.68445492   1.        ]
[119.85984468 346.90581799   1.        ]
[117.03484011 340.57907075   1.        ]
[118.52922273 342.09618664   1.        ]
[118.8823669  346.08836794   1.        ]
[120.03475857 347.51832628   1.        ]
[118.91659498 345.11133051   1.        ]
[116.65862513 340.87652659   1.        ]
[116.36346573 336.34817672   1.        ]
[113.40127683 325.85636055   1.        ]
[109.51778126 318.89726233   1.        ]
[107.03081131 313.33702421   1.        ]
[103.94852567 312.98177004   1.        ]
[100.19024469 31

[ 82.61250675 267.87675142   1.        ]
[ 83.63840368 267.33374882   1.        ]
[ 84.67490602 266.13968706   1.        ]
[ 83.43739939 262.95161319   1.        ]
[ 83.7616587 262.1521368   1.       ]
[ 83.05931282 263.35141373   1.        ]
[ 83.1321004  265.35405254   1.        ]
[ 82.96084058 263.76832557   1.        ]
[ 82.81518793 266.29499292   1.        ]
[ 84.0912765  266.40085697   1.        ]
[ 88.05694675 273.8404448    1.        ]
[ 91.26837969 276.84742296   1.        ]
[ 95.84529686 277.24310458   1.        ]
[101.66209221 273.75088644   1.        ]
[109.84745312 274.00630474   1.        ]
[112.94346738 277.2873404    1.        ]
[119.83975554 282.43789005   1.        ]
[126.57123947 281.59466887   1.        ]
[132.50616395 277.98600125   1.        ]
[143.0654726  272.63400602   1.        ]
[150.67810321 270.75892401   1.        ]
[164.14951438 269.04337883   1.        ]
[171.67978811 266.93458986   1.        ]
[181.38909149 263.61502528   1.        ]
[190.10270929 257.0

[ 86.43029118 149.6448791    1.        ]
[ 86.89676213 149.64531481   1.        ]
[ 87.10535192 149.39286137   1.        ]
[ 90.31438828 146.01710236   1.        ]
[ 99.37118572 144.7609055    1.        ]
[101.19919586 140.82886124   1.        ]
[104.40150928 140.49275589   1.        ]
[115.33708507 143.85887623   1.        ]
[120.65551376 143.99694729   1.        ]
[128.7798717  150.66230583   1.        ]
[133.51174319 153.32900429   1.        ]
[148.79324991 159.72469521   1.        ]
[152.16898108 162.62890935   1.        ]
[157.06127727 170.03739977   1.        ]
[168.70196724 177.02716184   1.        ]
[173.78031123 185.01314402   1.        ]
[182.57114792 194.21223998   1.        ]
[183.06458235 203.22673035   1.        ]
[190.11459541 216.75539875   1.        ]
[196.38256246 227.59041509   1.        ]
[198.12629652 233.13812685   1.        ]
[203.23169279 242.72770691   1.        ]
[208.03874111 252.02880859   1.        ]
[210.63165581 260.30830032   1.        ]
[211.51147044 26

[294.69535589 336.532336     1.        ]
[295.18071675 336.93055224   1.        ]
[296.80416632 338.32496202   1.        ]
[296.87430096 338.31743205   1.        ]
[297.2781539  336.74189067   1.        ]
[297.36828375 337.27049375   1.        ]
[300.86248779 337.88980007   1.        ]
[299.74305058 341.01695311   1.        ]
[302.3828392  342.13038754   1.        ]
[301.28027248 339.8470422    1.        ]
[302.57336664 340.62635618   1.        ]
[302.54937935 340.70449716   1.        ]
[301.65487862 341.09880161   1.        ]
[297.47992182 340.15187787   1.        ]
[292.13120914 334.31454992   1.        ]
[291.23386961 331.32413578   1.        ]
[279.88743567 330.92930841   1.        ]
[276.35399017 333.47327709   1.        ]
[270.90678406 338.32090688   1.        ]
[266.04909563 340.7424953    1.        ]
[264.80550289 343.04861832   1.        ]
[263.91892838 342.81423044   1.        ]
[263.97583151 341.54015887   1.        ]
[259.85161364 339.88306075   1.        ]
[248.15952349 33

In [ ]:
csvfile

In [ ]:
test = move_data
test

In [ ]:
import numpy as np
pd.options.mode.chained_assignment = None
p['coords'] = list(range(1,len(p)+1))
p.index = list(range(1,len(p)+1))

In [ ]:
p

In [ ]:
p = test[(test['y']>54) & (test['x']>38)]

In [ ]:
test = move_data['x'] - 38
test[test>0]

In [ ]:
def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out

def fieldOfView(data, filename, outputName, radius, rotationAngle, color, viewRange, thickness, displayVideo = False):
    '''    
    Task: draw sector for field of view

    All input parameters must be Int

    PARAMETERS:

    data: mouse movement that read from .h5 file, including head and tail. 

    angle: 0, rotation angle

    radius: radius of the sector, millimeter

    color: sector line color

    viewRange: range that the mouse has vision on

    thickness: the thickness of the sector line

    '''

    #Obtain coordinates data for head and tail, as well as the head direction by degrees
    

    head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
    tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
    degrees = head_dir(data)[1]
    
    #Get pixel to mm conversion
    x, y, lengthX, lengthY = getCoord(data)
    length = lengthX if lengthX < lengthY else lengthY
    convert = 440/length # video side length 440mm, square
    radius = round(radius / convert)
    
    
    #cap: Read video
    #out: Write video
    video = videoWriter("field of view", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        try:
            for center, angle in tqdm(zip(head, degrees)):
                ret,img = cap.read()
            #draw the sector representing the field of view of mouse
                angle = int(round(angle))
                startAngle = angle - viewRange
                endAngle = angle + viewRange
                img2 = drawSector(img, radius, center, rotationAngle, startAngle, endAngle, color, thickness)
                out.write(img2)
                    
                if displayVideo == True:
                    cv2.imshow('Draw field of view video',img2) 
                    
                # Break the loop when the last frame has been rotated
                if count == frameCnt:
                    print (filename, 'successfully ', "added field of view!!!")
                    return
                else:
                    count += 1

                k=cv2.waitKey(30) & 0xff
                #once you inter Esc capturing will stop
                if k==27:
                    break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [ ]:
BLACK = (0,0,0)
from math import degrees, atan2
def drawSector(image, radius, center, angle, startAngle, endAngle, color, thickness):
    # Ellipse parameters
    axes = (radius, radius)

    cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
    points = cv2.ellipse2Poly(center, axes, angle, startAngle, endAngle, thickness)
    point1 = tuple(points[0]) # End's coordinates
    point2 = tuple(points[-1]) # Another end's coordinates
    cv2.line(image, center, point1, color)
    cv2.line(image, center, point2, color)
    return image

def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)

In [ ]:
x = dataModifier(move_data)

In [ ]:
fieldOfView(x, '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest1.mp4", 50, 0, BLACK, 60, 1)

In [ ]:
x, y, lengthX, lengthY = getCoord(move_data)

In [ ]:
x, y, lengthX, lengthY

In [ ]:
convertX = 440/lengthX
convertY = 440/lengthY
print(convertX, convertY)

In [ ]:
440/331

In [ ]:
videoCropper('1034 SI_B, Aug 15, 13 7 49 rotated.mp4', "cropTest rotated.mp4", y, x, lengthY, lengthX)

In [ ]:
##Question: choose length of the edge in order to be converted in video processing